**Initial Data Cleaning: **

In this notebook, the data is cleaned and prepared for further analysis. The following steps are taken on order to arrive at the two datasets that are used in visualization and modeling:

1) Remove employees and keep only active and inactive customers (not prospects or former customers).

2) Remove "null group" customers which were found to have the same group of variables as null. This points to a potential systematic error with these records.

3) Create the number of product variable which is a count of all products not including credit card (target).

4) Remove clients with no products. 

5) Change data types and fill na's with 0 where necessary. 

6) Get only one row per customer (most current state of the customer not including churned relationships). 

7) Remove duplicates.

At this point, the data is exported to be used in the data visualization portion of the project (santander_data_visualization.ipynb).

8) Create a balanced sample by undersampling to select only as many negative outcomes of the target variable as we have positive ones. 

This dataset is then exported to be used for the modeling portion of the project (santander_model.ipynb). 



In [2]:
# Import necessary packages
import numpy as np # for numerical analysis
import pandas as pd # for data preparation and manipulation

# Import dataset - replace file path with your location, train_ver2 is the main  
# file available at https://www.kaggle.com/c/santander-product-recommendation/data
df_train = pd.read_csv('/content/drive/MyDrive/Santander/train_ver2.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# rename columns to english translations 

df_train.columns = ['date', 'customer_code', 'employee_index',
                    'customer_country_residence', 'customer_sex', 'age', 
                    'start_of_relationship', 'new_customer_6mo', 'vintage', 
                    'primary_cust_to_non', 'last_date_as_primary',
                    'customer_type', 'customer_relation_type', 
                    'residence_index', 'foreigner_index', 
                    'employee_spouse_index', 'channel_used_to_join',
                    'deceased_index', 'address_type', 'province_code',
                    'province_name', 'activity_index', 'gross_household_income',
                    'segment', 'savings_account', 'guarantees',
                    'current_accounts', 'derivada_account', 'payroll_account',
                    'junior_account', 'mas_particular_account',
                    'particular_account', 'particular_plus_account',
                    'short_term_deposits', 'medium_term_deposits',
                    'long_term_deposits', 'e_account', 'funds', 'mortgage',
                    'retirement', 'loans', 'taxes', 'credit_card', 'securities',
                    'home_account', 'payroll', 'pensions', 'direct_debit' ]

df_train.head()

,date,customer_code,employee_index,customer_country_residence,customer_sex,age,start_of_relationship,new_customer_6mo,vintage,primary_cust_to_non,last_date_as_primary,customer_type,customer_relation_type,residence_index,foreigner_index,employee_spouse_index,channel_used_to_join,deceased_index,address_type,province_code,province_name,activity_index,gross_household_income,segment,savings_account,guarantees,current_accounts,derivada_account,payroll_account,junior_account,mas_particular_account,particular_account,particular_plus_account,short_term_deposits,medium_term_deposits,long_term_deposits,e_account,funds,mortgage,retirement,loans,taxes,credit_card,securities,home_account,payroll,pensions,direct_debit
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,NaN,1,A,S,N,NaN,KHL,N,1.0,29.0,MALAGA,1.0,87218.10,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1,I,S,S,NaN,KHE,N,1.0,13.0,CIUDAD REAL,0.0,35548.74,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1,I,S,N,NaN,KHE,N,1.0,13.0,CIUDAD REAL,0.0,122179.11,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,NaN,1,I,S,N,NaN,KHD,N,1.0,50.0,ZARAGOZA,0.0,119775.54,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1,A,S,N,NaN,KHE,N,1.0,50.0,ZARAGOZA,1.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


In [ ]:
# Show percent of null values by column
df_train.isnull().mean()

date                          0.000000
customer_code                 0.000000
employee_index                0.002032
customer_country_residence    0.002032
customer_sex                  0.002037
age                           0.000000
start_of_relationship         0.002032
new_customer_6mo              0.002032
vintage                       0.000000
primary_cust_to_non           0.002032
last_date_as_primary          0.998183
customer_type                 0.010975
customer_relation_type        0.010975
residence_index               0.002032
foreigner_index               0.002032
employee_spouse_index         0.999868
channel_used_to_join          0.013638
deceased_index                0.002032
address_type                  0.002032
province_code                 0.006858
province_name                 0.006858
activity_index                0.002032
gross_household_income        0.204756
segment                       0.013876
savings_account               0.000000
guarantees               

In [ ]:
# Show column data type
df_train.dtypes

date                           object
customer_code                   int64
employee_index                 object
customer_country_residence     object
customer_sex                   object
age                            object
start_of_relationship          object
new_customer_6mo              float64
vintage                        object
primary_cust_to_non           float64
last_date_as_primary           object
customer_type                  object
customer_relation_type         object
residence_index                object
foreigner_index                object
employee_spouse_index          object
channel_used_to_join           object
deceased_index                 object
address_type                  float64
province_code                 float64
province_name                  object
activity_index                float64
gross_household_income        float64
segment                        object
savings_account                 int64
guarantees                      int64
current_acco

In [ ]:
# Remove employees from data
df_train =  df_train[df_train.employee_index == 'N']

# Keep only active or inactive customers
bool_series = df_train.customer_relation_type.isin(['A','I'])
df_train = df_train[bool_series]

# Remove "null group" columns - those with null values in these columns are null
# in several others. Removing these groups deals with many of the null values
df_train = df_train.dropna(subset=['customer_sex'])
df_train = df_train.dropna(subset=['customer_type'])

In [ ]:
# replace na's with zero
df_train = df_train.apply(lambda x: 
                          x.fillna(0) if x.name in ['payroll', 
                                                    'pensions', 
                                                    'employee_spouse_index'] 
                          else x) 

# Create Number of Products Column
df_train['number_of_products'] = (df_train['savings_account'] + 
                                  df_train['guarantees'] + 
                                  df_train['current_accounts'] + 
                                  df_train['derivada_account'] + 
                                  df_train['payroll_account']+ 
                                  df_train['junior_account'] + 
                                  df_train['mas_particular_account'] + 
                                  df_train['particular_account'] + 
                                  df_train['particular_plus_account'] + 
                                  df_train['short_term_deposits'] + 
                                  df_train['medium_term_deposits'] + 
                                  df_train['long_term_deposits'] + 
                                  df_train['e_account'] + df_train['funds'] + 
                                  df_train['mortgage'] + 
                                  df_train['retirement'] + 
                                  df_train['loans'] + 
                                  df_train['taxes'] + 
                                  df_train['securities'] + 
                                  df_train['home_account'] + 
                                  df_train['payroll'] + df_train['pensions'] + 
                                  df_train['direct_debit'] ) 

#Remove churned clients froim dataset (had 0 products)
df_train =  df_train[df_train.number_of_products > 0]

# Convert vintage to numeric variable
df_train["vintage"] = pd.to_numeric(df_train["vintage"], errors = 'coerce', 
                                    downcast = 'integer')

#Keep the last record for all customers 
df_train = df_train[df_train.groupby(
    'customer_code')['vintage'].transform('max') == df_train['vintage']]

#Remove any duplicate records
df_train.drop_duplicates(subset ="customer_code", keep = 'last', inplace = True)


In [ ]:
df_train.shape

(720741, 49)

In [ ]:
# Save "clean" dataset to csv, this file is used for data visualization
df_sample.to_csv('df_sample.csv')
!cp df_sample.csv '/content/drive/MyDrive/Santander'

In [ ]:
# Showing the distribution of the target variable shows a strong class imbalance
# in the target variable (credit_card)
df_train.credit_card.value_counts()

0    686806
1     33935
Name: credit_card, dtype: int64

In [ ]:
# Undersampling to create a balanced (and smaller) dataset

# Get the number of obs. with a positive target outcome
num_cc_cust = len(df_train[df_train['credit_card'] == 1])

# Get indices for customers with no credit card
no_cc_indices = df_train[df_train.credit_card == 0].index 

# Pick list of random indices of same number as there are positive obs. 
random_no_cc_indices = np.random.choice(no_cc_indices, 
                                        num_cc_cust, replace=False) 

 # Get indices of positive observations
cc_indices = df_train[df_train.credit_card == 1].index

# Combine indices of credit card and selected non credit card customers
under_sample_indices = np.concatenate([random_no_cc_indices, cc_indices]) 

# Sample balanced dataset from original dataset
df_sample = df_train.loc[under_sample_indices] 

In [ ]:
print(df_sample.shape) # New dataset size
print(df_sample.credit_card.value_counts()) # New distribution of target variable 

(67870, 49)
1    33935
0    33935
Name: credit_card, dtype: int64


In [ ]:
# Save undersampled dataset to csv, this file is used in modeling 
df_sample.to_csv('df_u_sample.csv')
!cp df_u_sample.csv '/content/drive/MyDrive/Santander'